In [4]:
class Tree:
    """
    >>> t = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
    >>> t.label
    3
    >>> t.branches[0].label
    2
    >>> t.branches[1].is_leaf()
    True
    """
    def __init__(self, label, branches=[]):
        for b in branches:
            assert isinstance(b, Tree)
        self.label = label
        self.branches = list(branches)

    def is_leaf(self):
        return not self.branches

    def map(self, fn):
        """
        Apply a function `fn` to each node in the tree and mutate the tree.

        >>> t1 = Tree(1)
        >>> t1.map(lambda x: x + 2)
        >>> t1.map(lambda x : x * 4)
        >>> t1.label
        12
        >>> t2 = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
        >>> t2.map(lambda x: x * x)
        >>> t2
        Tree(9, [Tree(4, [Tree(25)]), Tree(16)])
        """
        self.label = fn(self.label)
        for b in self.branches:
            b.map(fn)

    def __contains__(self, e):
        """
        Determine whether an element exists in the tree.

        >>> t1 = Tree(1)
        >>> 1 in t1
        True
        >>> 8 in t1
        False
        >>> t2 = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
        >>> 6 in t2
        False
        >>> 5 in t2
        True
        """
        if self.label == e:
            return True
        for b in self.branches:
            if e in b:
                return True
        return False

    def __repr__(self):
        if self.branches:
            branch_str = ', ' + repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(self.label, branch_str)

    def __str__(self):
        def print_tree(t, indent=0):
            tree_str = '  ' * indent + str(t.label) + "\n"
            for b in t.branches:
                tree_str += print_tree(b, indent + 1)
            return tree_str
        return print_tree(self).rstrip()


class Link:
    """A linked list.

    >>> s = Link(1)
    >>> s.first
    1
    >>> s.rest is Link.empty
    True
    >>> s = Link(2, Link(3, Link(4)))
    >>> s.first = 5
    >>> s.rest.first = 6
    >>> s.rest.rest = Link.empty
    >>> s                                    # Displays the contents of repr(s)
    Link(5, Link(6))
    >>> s.rest = Link(7, Link(Link(8, Link(9))))
    >>> s
    Link(5, Link(7, Link(Link(8, Link(9)))))
    >>> print(s)                             # Prints str(s)
    <5 7 <8 9>>
    """
    empty = ()

    def __init__(self, first, rest=empty):
        assert rest is Link.empty or isinstance(rest, Link)
        self.first = first
        self.rest = rest

    def __repr__(self):
        if self.rest is not Link.empty:
            rest_repr = ', ' + repr(self.rest)
        else:
            rest_repr = ''
        return 'Link(' + repr(self.first) + rest_repr + ')'

    def __str__(self):
        string = '<'
        while self.rest is not Link.empty:
            string += str(self.first) + ' '
            self = self.rest
        return string + str(self.first) + '>'


In [ ]:
def prune_min(t):
  while len(t.branches) > 1:
    largest = max(t.branches, key = lambda b: b.label)
    t.branches.remove(largest)
  for b in t.branches:
    prune_min(b)

t2 = Tree(6, [Tree(3), Tree(4)])
prune_min(t2)
t2

Tree(6, [Tree(3)])

In [ ]:
(define (reverse lst)
    (if (null? lst)
        lst
        (append (reverse (cdr lst)) (list (car lst)))))

(define (split-at lst n)
    (define (iter new old i)
        (if (or (null? old) (= i n))
            (cons (reverse new) old)
            (iter (cons (car old) new) (cdr old) (+ i 1))))
    (iter () lst 0)
)

(define nat '(1 2 3 4 5))

(split-at nat 2)

In [1]:
def align_skeleton(skeleton, code):
  skeleton, code = skeleton.replace(" ", ""), code.replace(" ", "")

  def helper_align(skeleton_idx, code_idx):
    if skeleton_idx == len(skeleton) and code_idx == len(code):
      return '', 0
    if skeleton_idx < len(skeleton) and code_idx == len(code):
      edits = "".join(["-[" + c + "]" for c in skeleton[skeleton_idx:]])
      return edits, len(skeleton) - skeleton_idx
    if skeleton_idx == len(skeleton) and code_idx < len(code):
      edits = "".join(["+[" + c + "]" for c in code[code_idx:]])
      return edits, len(code) - code_idx
          
    possibilities = []
    skel_char, code_char = skeleton[skeleton_idx], code[code_idx]
    # Match
    if skel_char == code_char:
      s, c = helper_align(skeleton_idx + 1, code_idx + 1)
      new_s = code_char + s
      possibilities.append((new_s, c))
    # Insert
    s, c = helper_align(skeleton_idx, code_idx + 1)
    new_s = '+[' + code_char + ']' + s
    possibilities.append((new_s, c + 1))
    # Delete
    s, c = helper_align(skeleton_idx + 1, code_idx)
    new_s = '-[' + skel_char + ']' + s
    possibilities.append((new_s, c + 1))
    return min(possibilities, key=lambda x: x[1])
  result, cost = helper_align(0, 0)
  return result

align_skeleton(skeleton="", code="i")

'+[i]'

In [2]:
def num_splits(l, d):
  def difference_sofar(s, n):
    if len(s) == 0:
      return 1 if abs(n) <= d else 0
    first, rest = s[0], s[1:]
    return difference_sofar(rest, n + first) + difference_sofar(rest, n - first)
  return difference_sofar(l, 0) // 2

num_splits([1, 5, 4], 0)

1

In [6]:
def insert(link, value, index):
  if index == 0:
    link.rest = Link(link.first, link.rest)
    link.first = value
  elif link.rest == Link.empty:
    raise IndexError
  else:
    insert(link.rest, value, index - 1)

link = Link(1, Link(2, Link(3)))
print(link)
insert(link, 9001, 0)
print(link)

<1 2 3>
<9001 1 2 3>


In [ ]:
(define-macro (switch expr cases)
    (cons 'cond
          (map (lambda (case) (cons (eq? (eval expr) (car case)) (cons expr (cdr case))))
               cases))    
)

(switch (+ 1 1) ((1 (print 'a))
                      (2 (print 'b))
                      (3 (print 'c))))